In [2]:
import pandas as pd
from bert import Ner
from tqdm import tqdm
from sklearn.metrics import classification_report
import numpy as np
import json

In [3]:
tqdm.pandas()

C:\Users\gabriel.oliveira.luz\AppData\Local\Continuum\anaconda3\envs\projetofinal2\lib\site-packages\tqdm\std.py:725: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
!pip list

In [3]:
df = pd.read_csv('data_test_ber/ner_dataset.csv', encoding="ISO-8859-1")  
df.head(5)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [4]:
df['sentence2'] = df.apply(lambda x: 0 if pd.isna(x['Sentence #']) else 1, axis=1)
df.head()

,Sentence #,Word,POS,Tag,sentence2
0,Sentence: 1,Thousands,NNS,O,1
1,NaN,of,IN,O,0
2,NaN,demonstrators,NNS,O,0
3,NaN,have,VBP,O,0
4,NaN,marched,VBN,O,0


In [5]:
df['sentence2'] = df['sentence2'].cumsum()

In [6]:
df.head()

,Sentence #,Word,POS,Tag,sentence2
0,Sentence: 1,Thousands,NNS,O,1
1,NaN,of,IN,O,1
2,NaN,demonstrators,NNS,O,1
3,NaN,have,VBP,O,1
4,NaN,marched,VBN,O,1


In [7]:
de_para = {
    'B-gpe': 'B-MISC',
    'I-gpe': 'I-MISC',
    'O-gpe': 'O-MISC',
    'B-tim': 'B-MISC',
    'I-tim': 'I-MISC',
    'O-tim': 'O-MISC',
    'B-art': 'B-MISC',
    'I-art': 'I-MISC',
    'O-art': 'O-MISC',
    'B-eve': 'B-MISC',
    'I-eve': 'I-MISC',
    'O-eve': 'O-MISC',
    'B-nat': 'B-MISC',
    'I-nat': 'I-MISC',
    'O-nat': 'O-MISC',
    'B-geo':'B-LOC',
    'I-geo':'I-LOC',
    'O-geo':'O-LOC',
    'B-org':'B-ORG',
    'I-org':'I-ORG',
    '0-org':'0-ORG',
    'B-per':'B-PER',
    'I-per':'I-PER',
    'O-per':'O-PER'
}

In [8]:
df['Tag'] = df['Tag'].replace(de_para)

In [9]:
df.head(10)

,Sentence #,Word,POS,Tag,sentence2
0,Sentence: 1,Thousands,NNS,O,1
1,NaN,of,IN,O,1
2,NaN,demonstrators,NNS,O,1
3,NaN,have,VBP,O,1
4,NaN,marched,VBN,O,1
5,NaN,through,IN,O,1
6,NaN,London,NNP,B-LOC,1
7,NaN,to,TO,O,1
8,NaN,protest,VB,O,1
9,NaN,the,DT,O,1


In [10]:
model = Ner("out_base/")

In [11]:
out = model.predict('I live in Brazil')
print(out)

[{'word': 'I', 'tag': 'O', 'confidence': 0.9991158843040466}, {'word': 'live', 'tag': 'O', 'confidence': 0.9997305274009705}, {'word': 'in', 'tag': 'O', 'confidence': 0.9997486472129822}, {'word': 'Brazil', 'tag': 'B-LOC', 'confidence': 0.9995967745780945}]


In [12]:
df_to_model = df.groupby(['sentence2'])['Word'].apply(' '.join).reset_index()

In [13]:
df_to_model.head()

,sentence2,Word
0,1,Thousands of demonstrators have marched throug...
1,2,Families of soldiers killed in the conflict jo...
2,3,They marched from the Houses of Parliament to ...
3,4,"Police put the number of marchers at 10,000 wh..."
4,5,The protest comes on the eve of the annual con...


In [14]:
df_to_model = df_to_model.iloc[:200]

In [15]:
df_to_model.count

<bound method DataFrame.count of      sentence2                                               Word
0            1  Thousands of demonstrators have marched throug...
1            2  Families of soldiers killed in the conflict jo...
2            3  They marched from the Houses of Parliament to ...
3            4  Police put the number of marchers at 10,000 wh...
4            5  The protest comes on the eve of the annual con...
..         ...                                                ...
195        196  The United Nations says December 's Indian Oce...
196        197  The U.N. 's Food and Agricultural Organization...
197        198  It said the loss was significant in a region w...
198        199  The FAO 's estimate includes damage to fishing...
199        200  The agency says it has sent experts to help re...

[200 rows x 2 columns]>

In [16]:
df_to_model['result'] = df_to_model.progress_apply(lambda x: model.predict(x['Word']), axis=1)

100%|██████████| 200/200 [01:28<00:00,  2.26it/s]


In [17]:
df_to_model.head(10)

,sentence2,Word,result
0,1,Thousands of demonstrators have marched throug...,"[{'word': 'Thousands', 'tag': 'O', 'confidence..."
1,2,Families of soldiers killed in the conflict jo...,"[{'word': 'Families', 'tag': 'O', 'confidence'..."
2,3,They marched from the Houses of Parliament to ...,"[{'word': 'They', 'tag': 'O', 'confidence': 0...."
3,4,"Police put the number of marchers at 10,000 wh...","[{'word': 'Police', 'tag': 'O', 'confidence': ..."
4,5,The protest comes on the eve of the annual con...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9..."
5,6,The party is divided over Britain 's participa...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9..."
6,7,The London march came ahead of anti-war protes...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9..."
7,8,The International Atomic Energy Agency is to h...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9..."
8,9,Iran this week restarted parts of the conversi...,"[{'word': 'Iran', 'tag': 'B-LOC', 'confidence'..."
9,10,Iranian officials say they expect to get acces...,"[{'word': 'Iranian', 'tag': 'B-MISC', 'confide..."


In [18]:
df_ent = df[df['Tag'] != 'O']
df_ent.head()

,Sentence #,Word,POS,Tag,sentence2
6,NaN,London,NNP,B-LOC,1
12,NaN,Iraq,NNP,B-LOC,1
18,NaN,British,JJ,B-MISC,1
42,NaN,Bush,NNP,B-PER,2
65,NaN,Hyde,NNP,B-LOC,3


In [19]:
df_ent['dict'] = df_ent.apply(lambda x: {x['Word']: x['Tag']}, axis=1)
df_ent.head()

C:\Users\gabriel.oliveira.luz\AppData\Local\Continuum\anaconda3\envs\projetofinal2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Sentence #,Word,POS,Tag,sentence2,dict
6,NaN,London,NNP,B-LOC,1,{'London': 'B-LOC'}
12,NaN,Iraq,NNP,B-LOC,1,{'Iraq': 'B-LOC'}
18,NaN,British,JJ,B-MISC,1,{'British': 'B-MISC'}
42,NaN,Bush,NNP,B-PER,2,{'Bush': 'B-PER'}
65,NaN,Hyde,NNP,B-LOC,3,{'Hyde': 'B-LOC'}


In [20]:
#df_ent.groupby(['sentence2'])['dict'].apply(lambda x: list(x))#.reset_index()
df_list = df_ent.groupby(['sentence2'])['dict'].apply(lambda x: list(x))#.reset_index()
df_list.head(10)

sentence2
1     [{'London': 'B-LOC'}, {'Iraq': 'B-LOC'}, {'Bri...
2                                   [{'Bush': 'B-PER'}]
3                [{'Hyde': 'B-LOC'}, {'Park': 'I-LOC'}]
5     [{'Britain': 'B-LOC'}, {'Labor': 'B-ORG'}, {'P...
6     [{'Britain': 'B-MISC'}, {'Iraq': 'B-LOC'}, {'B...
7     [{'London': 'B-LOC'}, {'Rome': 'B-LOC'}, {'Par...
8     [{'International': 'B-ORG'}, {'Atomic': 'I-ORG...
9            [{'Iran': 'B-MISC'}, {'Isfahan': 'B-LOC'}]
10    [{'Iranian': 'B-MISC'}, {'Wednesday': 'B-MISC'...
12    [{'European': 'B-ORG'}, {'Union': 'I-ORG'}, {'...
Name: dict, dtype: object

In [21]:
#df_final = pd.merge(df_to_model, df_list.loc[:,['sentence2','list']], how='left', on='sentence2') 
df_final = pd.merge(df_to_model, df_list, how='left', on='sentence2') 
df_final.head(10)

,sentence2,Word,result,dict
0,1,Thousands of demonstrators have marched throug...,"[{'word': 'Thousands', 'tag': 'O', 'confidence...","[{'London': 'B-LOC'}, {'Iraq': 'B-LOC'}, {'Bri..."
1,2,Families of soldiers killed in the conflict jo...,"[{'word': 'Families', 'tag': 'O', 'confidence'...",[{'Bush': 'B-PER'}]
2,3,They marched from the Houses of Parliament to ...,"[{'word': 'They', 'tag': 'O', 'confidence': 0....","[{'Hyde': 'B-LOC'}, {'Park': 'I-LOC'}]"
3,4,"Police put the number of marchers at 10,000 wh...","[{'word': 'Police', 'tag': 'O', 'confidence': ...",NaN
4,5,The protest comes on the eve of the annual con...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9...","[{'Britain': 'B-LOC'}, {'Labor': 'B-ORG'}, {'P..."
5,6,The party is divided over Britain 's participa...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9...","[{'Britain': 'B-MISC'}, {'Iraq': 'B-LOC'}, {'B..."
6,7,The London march came ahead of anti-war protes...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9...","[{'London': 'B-LOC'}, {'Rome': 'B-LOC'}, {'Par..."
7,8,The International Atomic Energy Agency is to h...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9...","[{'International': 'B-ORG'}, {'Atomic': 'I-ORG..."
8,9,Iran this week restarted parts of the conversi...,"[{'word': 'Iran', 'tag': 'B-LOC', 'confidence'...","[{'Iran': 'B-MISC'}, {'Isfahan': 'B-LOC'}]"
9,10,Iranian officials say they expect to get acces...,"[{'word': 'Iranian', 'tag': 'B-MISC', 'confide...","[{'Iranian': 'B-MISC'}, {'Wednesday': 'B-MISC'..."


In [4]:
dados = pd.read_csv('dados_predict.csv', encoding="ISO-8859-1")  

In [5]:
def allocate_columns_for_confusion_matrix(df):
    data = df['dict'].to_dict()
    gt_entity_list = []
    for ind, label in tqdm(data.items()):
        if pd.isna(label) == False:
        #if label != 'nan':
            ground_truth = json.loads(label.replace("'", '"'))
            for label in ground_truth:
                word, entity = list(label.items())[0]
                gt_entity_list.append(entity)
    gt_entity_list = list(set(gt_entity_list))
    #gt_entity_list.append('O')
    matrix_cols = ['TP', 'FP', 'TN', 'FN']
    for col in matrix_cols:
        for entity in gt_entity_list:
            df[f'{col} - {entity}'] = 0
    return df

In [6]:
def generate_confusion_matrix_metrics2(row, confidence_threshold=0.9):
    gt_entity_list = []; gt_word_list = []
    if pd.isna(row['dict']) == False:
        ground_truth = json.loads(row['dict'].replace("'", '"'))
    else:
        ground_truth = []
#     import ipdb; ipdb.set_trace()
    predictions = json.loads(row['result'])
    for label in ground_truth:
        if pd.isna(label) == False:
            word, entity = list(label.items())[0]
            gt_entity_list.append(entity)
            gt_word_list.append(word)
    predicted_words = []; predicted_entities = []
    for result in predictions:
        if result['confidence'] >= confidence_threshold:
            predicted_word = result['word']
            predicted_entity = result['tag']
            if predicted_entity == 'O':
                continue
            predicted_words.append(predicted_word)
            predicted_entities.append(predicted_entity)
            if predicted_word in gt_word_list:
                word_index = gt_word_list.index(predicted_word)
                if predicted_entity == gt_entity_list[word_index]:
                    row[f"TP - {predicted_entity}"] += 1
                else:
                    row[f"FP - {predicted_entity}"] += 1
            else:
                predicted_entity = result['tag']
                row[f"FP - {predicted_entity}"] += 1
    for label in ground_truth:
        word, entity = list(label.items())[0]
        if word not in predicted_words:
            row[f"TN - {entity}"] += 1
        else:
            row[f"FN - {entity}"] += 1
    return row

In [26]:
#dados = dados.dropna()
#dados.head(5)

,sentence2,Word,result,dict
0,1,Thousands of demonstrators have marched throug...,"[{'word': 'Thousands', 'tag': 'O', 'confidence...","[{'London': 'B-LOC'}, {'Iraq': 'B-LOC'}, {'Bri..."
1,2,Families of soldiers killed in the conflict jo...,"[{'word': 'Families', 'tag': 'O', 'confidence'...",[{'Bush': 'B-PER'}]
2,3,They marched from the Houses of Parliament to ...,"[{'word': 'They', 'tag': 'O', 'confidence': 0....","[{'Hyde': 'B-LOC'}, {'Park': 'I-LOC'}]"
4,5,The protest comes on the eve of the annual con...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9...","[{'Britain': 'B-LOC'}, {'Labor': 'B-ORG'}, {'P..."
5,6,The party is divided over Britain 's participa...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9...","[{'Britain': 'B-MISC'}, {'Iraq': 'B-LOC'}, {'B..."


In [7]:
data = allocate_columns_for_confusion_matrix(dados)

100%|██████████| 200/200 [00:00<00:00, 40025.80it/s]


In [8]:
data.head(5)

,sentence2,Word,result,dict,TP - I-LOC,TP - B-LOC,TP - B-PER,TP - I-ORG,TP - B-MISC,TP - B-ORG,...,TN - I-PER,TN - I-MISC,FN - I-LOC,FN - B-LOC,FN - B-PER,FN - I-ORG,FN - B-MISC,FN - B-ORG,FN - I-PER,FN - I-MISC
0,1,Thousands of demonstrators have marched throug...,"[{'word': 'Thousands', 'tag': 'O', 'confidence...","[{'London': 'B-LOC'}, {'Iraq': 'B-LOC'}, {'Bri...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Families of soldiers killed in the conflict jo...,"[{'word': 'Families', 'tag': 'O', 'confidence'...",[{'Bush': 'B-PER'}],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,They marched from the Houses of Parliament to ...,"[{'word': 'They', 'tag': 'O', 'confidence': 0....","[{'Hyde': 'B-LOC'}, {'Park': 'I-LOC'}]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,"Police put the number of marchers at 10,000 wh...","[{'word': 'Police', 'tag': 'O', 'confidence': ...",NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,The protest comes on the eve of the annual con...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9...","[{'Britain': 'B-LOC'}, {'Labor': 'B-ORG'}, {'P...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
def generate_confusion_matrix_metrics(row, confidence_threshold=0.9):
    gt_entity_list = []; gt_word_list = []
    if pd.isna(row['dict']) == False:
        ground_truth = json.loads(row['dict'].replace("'", '"'))
        #print(ground_truth)
    else:
        ground_truth = []
        predictions = json.loads(row['result'].replace("'", '"'))
        #print(predictions)
    for label in ground_truth:
        if pd.isna(label) == False:
            #pass
            word, entity = list(label.items())[0]
            #gt_entity_list.append(entity)
            #gt_word_list.append(word)
    #predicted_words = []; predicted_entities = []
        

In [79]:
data2 = data.progress_apply(lambda x: generate_confusion_matrix_metrics(x), axis=1)

 38%|███▊      | 77/200 [00:00<00:00, 8556.40it/s]


JSONDecodeError: Expecting ',' delimiter: line 1 column 328 (char 327)

In [34]:
df_final.to_csv(r'C:\Users\gabriel.oliveira.luz\Documents\projetofinal_git\projetofinal\dados_predict.csv', index = False, header=True)

In [23]:
df_final.iloc[0,2:4]

result    [{'word': 'Thousands', 'tag': 'O', 'confidence...
dict      [{'London': 'B-LOC'}, {'Iraq': 'B-LOC'}, {'Bri...
Name: 0, dtype: object

In [30]:
y_true = df_final['dict']
y_pred = df_final['result']
target_names = ['B-MISC','I-MISC','O-MISC',
                'B-PER','I-PER', 'O-PER',
                'B-ORG','I-ORG','0-ORG',
                'B-LOC','I-LOC','O-LOC']

In [31]:
y_true = y_true.to_numpy()
y_true

array([list([{'London': 'B-LOC'}, {'Iraq': 'B-LOC'}, {'British': 'B-MISC'}]),
       list([{'Bush': 'B-PER'}]),
       list([{'Hyde': 'B-LOC'}, {'Park': 'I-LOC'}]), nan,
       list([{'Britain': 'B-LOC'}, {'Labor': 'B-ORG'}, {'Party': 'I-ORG'}, {'English': 'B-MISC'}, {'Brighton': 'B-LOC'}]),
       list([{'Britain': 'B-MISC'}, {'Iraq': 'B-LOC'}, {'British': 'B-MISC'}]),
       list([{'London': 'B-LOC'}, {'Rome': 'B-LOC'}, {'Paris': 'B-LOC'}, {'Madrid': 'B-LOC'}]),
       list([{'International': 'B-ORG'}, {'Atomic': 'I-ORG'}, {'Energy': 'I-ORG'}, {'Agency': 'I-ORG'}, {'Vienna': 'B-LOC'}, {'Wednesday': 'B-MISC'}, {'Iran': 'B-MISC'}]),
       list([{'Iran': 'B-MISC'}, {'Isfahan': 'B-LOC'}]),
       list([{'Iranian': 'B-MISC'}, {'Wednesday': 'B-MISC'}, {'IAEA': 'B-ORG'}]),
       nan,
       list([{'European': 'B-ORG'}, {'Union': 'I-ORG'}, {'U.S.': 'B-MISC'}, {'Iran': 'B-MISC'}, {'U.N.': 'B-ORG'}, {'Security': 'I-ORG'}, {'Council': 'I-ORG'}, {'Tehran': 'B-MISC'}, {'Nuclear': 'B-MISC'}, {'N

In [26]:
y_pred.to_numpy()

array([list([{'word': 'Thousands', 'tag': 'O', 'confidence': 0.9999738931655884}, {'word': 'of', 'tag': 'O', 'confidence': 0.9999744892120361}, {'word': 'demonstrators', 'tag': 'O', 'confidence': 0.999970555305481}, {'word': 'have', 'tag': 'O', 'confidence': 0.9999747276306152}, {'word': 'marched', 'tag': 'O', 'confidence': 0.9999754428863525}, {'word': 'through', 'tag': 'O', 'confidence': 0.9999703168869019}, {'word': 'London', 'tag': 'B-LOC', 'confidence': 0.999554455280304}, {'word': 'to', 'tag': 'O', 'confidence': 0.9999744892120361}, {'word': 'protest', 'tag': 'O', 'confidence': 0.9999734163284302}, {'word': 'the', 'tag': 'O', 'confidence': 0.9999730587005615}, {'word': 'war', 'tag': 'O', 'confidence': 0.9999626874923706}, {'word': 'in', 'tag': 'O', 'confidence': 0.9999629259109497}, {'word': 'Iraq', 'tag': 'B-LOC', 'confidence': 0.9997662901878357}, {'word': 'and', 'tag': 'O', 'confidence': 0.9999744892120361}, {'word': 'demand', 'tag': 'O', 'confidence': 0.9999728202819824}, {'w

In [ ]:
y_pred_flat = [val for sublist in y_pred for val in sublist]
y_pred_flat

In [ ]:
y_true_flat = [val for sublist in y_pred for val in sublist]
y_true_flat

In [ ]:
print(classification_report(y_true_flat, y_pred_flat, target_names=target_names))